# Unknown classes

In [2]:
import numpy as np
import os
from pathlib import Path
import cv2
import pandas as pd
import tqdm
from imutils import build_montages

def extractLine(line, h, w):
    c, *xyxy = [float(x) for x in line.strip().split()]
    c = int(c)
    # print(line)
    # print(c)
    # print(xyxy)
    cx = xyxy[0]*w
    cy = xyxy[1]*h
    bw = xyxy[2]*w
    bh = xyxy[3]*h
    xmin = int(np.ceil(cx - bw/2))
    xmax = int(np.ceil(cx + bw/2))
    ymin = int(np.ceil(cy - bh/2))
    ymax = int(np.ceil(cy + bh/2))

    return c, xmin, xmax, ymin, ymax

def txt2df(datasetPath):
    df = pd.DataFrame(columns=['img', 'class', 'xmin', 'xmax', 'ymin', 'ymax'])
        
    labels_dir = Path(f'{datasetPath}/labels')

    for txt in tqdm.tqdm(os.listdir(labels_dir)):
        # img = cv2.imread(
        #             os.path.join(Path(f'{datasetPath}/images'),
        #             txt.replace('.txt','.jpg')
        #                         )
        #                     )
        # h, w = img.shape[:2]
        h, w = 480, 640
        # print(f'{h}x{w}')

        with open(Path(labels_dir).joinpath(txt)) as f:
            lines = f.readlines()

            for i, line in enumerate(lines):
                c, xmin, xmax, ymin, ymax = extractLine(line, h, w)

                df_row = pd.DataFrame(
                            {
                            'img': [txt],
                            'class': [c],
                            'xmin': [xmin],
                            'xmax': [xmax],
                            'ymin': [ymin],
                            'ymax': [ymax]
                            }
                        )

                df = pd.concat([df,df_row], ignore_index=True)

    csv_suffix = f'_{Path(datasetPath).name}'
    df.to_csv(f'df{csv_suffix}.csv', index=False)
    print('Done!')

In [3]:
def buildMontage(df, imgRoot, show=False, save=False, exp=None):
    images = []

    for imgPath in tqdm.tqdm(df.img):
        try:
            # print(imgPath)
            image = cv2.imread(os.path.join(imgRoot, imgPath.replace('txt', 'jpg')))
            # print(image)
            imgRow = df[df['img'] == imgPath]
            xmin = int(imgRow.xmin)
            ymin = int(imgRow.ymin)
            xmax = int(imgRow.xmax)
            ymax = int(imgRow.ymax)
            image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (0,255,0), 3)
            images.append(image)
        except:
            image = cv2.imread(os.path.join(imgRoot, imgPath.replace('txt', 'jpg')))
            # print(image)
            imgRow = df[df['img'] == imgPath]
            imgRows = imgRow.values.tolist()
            for imgRow in imgRows:
                # print(imgRow)
                xmin = int(imgRow[4])
                xmax = int(imgRow[5])
                ymin = int(imgRow[6])
                ymax = int(imgRow[7])
                image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (0,255,0), 3)
            images.append(image)
            # pass

    montages = build_montages(images, (640//3, 640//4), (5,5))

    if show:
        showMontages(montages)

    if save:
        saveMontages(montages, exp)

    return montages

def showMontages(montages):
    for montage in montages:
        cv2.imshow('montage', montage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

def saveMontages(montages, exp):
    if not exp:
        print("If you pass save as argument, please provide experiment name so the script can generate a path and save.")
    
    path = f'C:\\Users\\leofi\\OneDrive - Universidade de Lisboa\\Documents\\GitHub\\yolov5\\yolov5\\runs\\detect\\{exp}\\montages'

    if not os.path.isdir(path):
        os.mkdir(os.path.isdir(path))

    for idx, montage in enumerate(montages):
        montage_name = f'montage_{idx}.png'
        cv2.imwrite(os.path.join(path, montage_name), montage)

In [4]:
df_pred = pd.read_csv('bg_df_hard_uc_hybrid1.csv')
df_pred.img.nunique()
df_pred

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.623393,0.625797,41,127,179,269
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.847749,0.850266,189,287,145,215
2,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.623393,0.625797,41,127,179,269
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.847749,0.850266,189,287,145,215
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.792853,0.796728,35,118,187,277
...,...,...,...,...,...,...,...,...
5741,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.676512,0.678031,174,219,157,209
5742,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.543904,0.545039,181,236,169,220
5743,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.738834,0.739920,273,328,209,256
5744,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.443241,0.443734,238,292,233,272


In [5]:
df_label = pd.read_csv('df_unknown_classes.csv')
# df_label[df_label.cls_conf==1].img.nunique()
df_label

,img,class,xmin,xmax,ymin,ymax
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,339,395,17,54
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,334,378,62,104
2,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,190,286,149,217
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,39,121,182,269
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,339,395,17,54
...,...,...,...,...,...,...
5951,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,8,286,338,187,233
5952,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,8,274,326,209,254
5953,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,12,186,228,168,216
5954,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,12,236,292,236,270


In [6]:
df_pred[df_pred.img.isin(df_label.img.unique())].shape[0] == df_pred.shape[0]

True

In [7]:
df_pred[df_pred.img.isin(df_label.img.unique())]

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.623393,0.625797,41,127,179,269
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.847749,0.850266,189,287,145,215
2,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.623393,0.625797,41,127,179,269
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.847749,0.850266,189,287,145,215
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.792853,0.796728,35,118,187,277
...,...,...,...,...,...,...,...,...
5741,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.676512,0.678031,174,219,157,209
5742,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.543904,0.545039,181,236,169,220
5743,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.738834,0.739920,273,328,209,256
5744,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.443241,0.443734,238,292,233,272


In [8]:
pathLabels = 'C:\\Users\\leofi\\OneDrive - Universidade de Lisboa\\Documents\\GitHub\\masters\\Data\\fc2015_yolo_uc_hard\\unknown_classes\\labels'
pathImages = pathLabels.replace('labels', 'images')
nameImages = sorted([f[:-len('.jpg')] for f in os.listdir(pathImages)])
nameLabelOG = sorted([f[:-len('.txt')] for f in os.listdir(pathLabels)])
namePred = sorted([f[:-len('.txt')] for f in df_pred.img.unique()])
nameLabels = sorted([f[:-len('.txt')] for f in df_label.img.unique()])

print(nameLabelOG==nameImages)
print(namePred==nameImages)
print(nameLabels==nameImages)
print(namePred==nameLabels)

namePredInImages = [f for f in namePred if f in nameImages]

print(len(namePred)==len(namePredInImages))

True
False
True
False
True


In [9]:
def box_iou(box1,box2):
    inter_w = abs((min(box1[2], box2[2]) - max(box1[0], box2[0])))
    inter_h = abs((min(box1[3], box2[3]) - max(box1[1], box2[1])))

    inter = inter_w*inter_h

    a1 = abs((box1[2]-box1[0])*(box1[3]-box1[1]))
    a2 = abs((box2[2]-box2[0])*(box2[3]-box2[1]))
    union = a1 + a2 - inter + 1e-12 #eps to avoid divide by 0 error

    return max(0, inter/union)

In [10]:
iou_th = 0.25
# print(tp_df.shape)
df_pred_iou = df_pred.copy()
df_pred_iou['iou'] = np.nan
df_pred_iou['label'] = np.nan
# Iterate through unique image IDs
for img in tqdm.tqdm(df_pred.img.unique()):
    df_pred_sub = df_pred[df_pred.img == img]
    df_label_sub = df_label[df_label.img == img].reset_index()

    # Calculate IoU for each pair of prediction and label bounding boxes in a vectorized manner
    pred_boxes = df_pred_sub[['xmin', 'ymin', 'xmax', 'ymax']].values
    label_boxes = df_label_sub[['xmin', 'ymin', 'xmax', 'ymax']].values

    for idx, pred_box in enumerate(pred_boxes):
        ious=[]
        for label_box in label_boxes:
            ious.append(box_iou(pred_box, label_box))
            # Update the 'iou' column with the maximum IoU for each prediction
        max_ious = np.max(ious)
        max_ious_index = ious.index(max_ious)
        # print(ious)
        # print(max_ious)
        # print(idx)
        # print(df_pred_sub[0])
        # print(df_pred_sub.index[idx])
        df_pred_iou.at[df_pred_sub.index[idx], 'iou'] = max_ious
        df_pred_iou.at[df_pred_sub.index[idx], 'label'] = df_label_sub['class'][max_ious_index]
        # print(df_label_sub['class'][max_ious_index])

# Filter out predictions based on the IoU threshold
df_pred_iou = df_pred_iou[df_pred_iou['iou'] >= iou_th]

# Remove the 'iou' column, if needed
# df_pred_iou = df_pred_iou.drop(columns=['iou'])

100%|██████████| 2753/2753 [00:14<00:00, 194.40it/s]


In [11]:
df_pred_iou

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax,iou,label
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.623393,0.625797,41,127,179,269,0.879454,3.0
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.847749,0.850266,189,287,145,215,1.126667,5.0
2,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.623393,0.625797,41,127,179,269,0.879454,3.0
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.847749,0.850266,189,287,145,215,1.126667,5.0
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.792853,0.796728,35,118,187,277,0.871348,3.0
...,...,...,...,...,...,...,...,...,...,...
5741,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.676512,0.678031,174,219,157,209,0.861538,12.0
5742,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.543904,0.545039,181,236,169,220,0.693361,12.0
5743,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.738834,0.739920,273,328,209,256,0.905222,8.0
5744,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.443241,0.443734,238,292,233,272,0.844526,12.0


In [12]:
imgPath = 'C:\\Users\\leofi\\OneDrive - Universidade de Lisboa\\Documents\\GitHub\\masters\\Data\\fc2015_yolo_uc_hard\\unknown_classes\\images'

buildMontage(df_pred_iou, imgPath, save=True, exp='hard_uc_hybrid')

100%|██████████| 5493/5493 [01:15<00:00, 72.87it/s] 


[array([[[ 24,  17,   2],
         [ 23,  13,   0],
         [ 93,  90,  59],
         ...,
         [241, 208, 162],
         [146, 126,  91],
         [ 45,  31,   3]],
 
        [[ 11,   0,   0],
         [ 47,  29,  28],
         [170, 157, 155],
         ...,
         [241, 208, 162],
         [146, 126,  91],
         [ 45,  30,   4]],
 
        [[ 44,  22,  41],
         [ 53,  30,  45],
         [ 21,   0,  22],
         ...,
         [239, 209, 160],
         [145, 126,  93],
         [ 42,  28,   9]],
 
        ...,
 
        [[  0,   2,   0],
         [  3,   5,  13],
         [ 57,  58,  79],
         ...,
         [109, 103, 128],
         [ 45,  30,  51],
         [ 24,   0,  18]],
 
        [[  2,   4,   4],
         [  0,   2,  10],
         [ 39,  40,  60],
         ...,
         [ 40,  34,  59],
         [ 13,   0,  17],
         [ 23,   0,  17]],
 
        [[ 18,  22,  17],
         [  0,   1,   4],
         [ 18,  20,  38],
         ...,
         [ 87,  81, 104],
  

In [13]:
# lowIouIdxs = np.arange(df_pred.shape[0]).tolist().remove(df_pred_iou.index.tolist())
lowIouIdxs = [item for item in df_pred.index if item not in df_pred_iou.index]
df_pred_lowIou = df_pred.iloc[lowIouIdxs]

In [14]:
buildMontage(df_pred.iloc[lowIouIdxs], imgPath, show=True)

100%|██████████| 253/253 [00:02<00:00, 109.94it/s]


[array([[[ 19,  21,   1],
         [ 19,  21,   0],
         [ 85,  89,  53],
         ...,
         [255, 237, 205],
         [103,  88,  79],
         [  7,   0,   5]],
 
        [[  5,   0,   0],
         [ 37,  31,  26],
         [164, 158, 151],
         ...,
         [255, 250, 218],
         [ 99,  87,  81],
         [ 14,   7,  14]],
 
        [[ 37,  26,  36],
         [ 41,  30,  40],
         [ 14,   0,  13],
         ...,
         [255, 253, 219],
         [101,  85,  78],
         [ 23,   9,  15]],
 
        ...,
 
        [[  0,   1,   0],
         [  0,   3,  10],
         [ 41,  55,  77],
         ...,
         [ 90, 104, 126],
         [ 21,  35,  41],
         [  0,   3,   0]],
 
        [[  3,   1,   0],
         [  0,   2,   7],
         [ 26,  40,  63],
         ...,
         [ 22,  35,  57],
         [  0,   1,   8],
         [  1,   8,   5]],
 
        [[ 23,  18,  19],
         [  0,   0,   7],
         [  2,  19,  40],
         ...,
         [ 69,  83, 105],
  

In [15]:
df_pred_iou.to_csv('hard_onUnknownSp_highIou.csv')
df_pred_lowIou.to_csv('hard_onUnknownSp_lowIou.csv')

# Test subset

In [25]:
df_pred = pd.read_csv('bg_df_hard_test_hybrid.csv')
df_pred

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.664786,0.668394,359,497,245,342
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.374855,0.377023,474,555,215,298
2,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.274862,0.276366,9,62,130,193
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.708479,0.709997,377,428,56,122
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,2,0.257919,0.269762,21,78,131,213
...,...,...,...,...,...,...,...,...
1046,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.595292,0.629056,141,205,104,146
1047,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.563304,0.639010,53,115,77,122
1048,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.433499,0.438391,14,63,47,90
1049,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.612968,0.614991,416,461,93,135


In [18]:
df_label = pd.read_csv('df_hard_gt.csv')
# df_label[df_label.cls_conf==1].img.nunique()
df_label

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,NaN,NaN,467,557,210,300
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,NaN,NaN,376,428,58,125
2,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,NaN,NaN,387,439,86,147
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,NaN,NaN,387,439,86,147
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,NaN,NaN,388,446,94,157
...,...,...,...,...,...,...,...,...
1019,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,1,NaN,NaN,446,514,105,147
1020,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,9,NaN,NaN,242,294,240,288
1021,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,9,NaN,NaN,256,302,244,292
1022,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,5,NaN,NaN,416,460,93,135


In [32]:
print(([img for img in df_pred.img.unique() if img not in df_label.img.unique()]))

['01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame1.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame195.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame225.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame231.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame234.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame238.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame239.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame240.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame241.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame3.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame327.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame352.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame353.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame39.txt', '01465f8f61db58564cd37ce2dfc519c5#201106090830_0_frame41.txt', '01465f8f61db58564cd37ce2dfc519c5#20110609083

In [23]:
df_pred[df_pred.img.isin(df_label.img.unique())].shape[0]

857

In [20]:
pathLabels = 'C:\\Users\\leofi\\OneDrive - Universidade de Lisboa\\Documents\\GitHub\\masters\\Data\\fc2015_yolo_uc_hard\\test\\labels'
pathImages = pathLabels.replace('labels', 'images')
nameImages = sorted([f[:-len('.jpg')] for f in os.listdir(pathImages)])
nameLabelOG = sorted([f[:-len('.txt')] for f in os.listdir(pathLabels)])
namePred = sorted([f[:-len('.txt')] for f in df_pred.img.unique()])
nameLabels = sorted([f[:-len('.txt')] for f in df_label.img.unique()])

print(nameLabelOG==nameImages)
print(namePred==nameImages)
print(nameLabels==nameImages)
print(namePred==nameLabels)

namePredInImages = [f for f in namePred if f in nameImages]

print(len(namePred)==len(namePredInImages))

False
False
False
False
True


In [43]:
iou_th = 0.25
# print(tp_df.shape)
df_pred_iou = df_pred.copy()
df_pred_iou['iou'] = np.nan
df_pred_iou['label'] = np.nan
# Iterate through unique image IDs
for img in tqdm.tqdm(df_label.img.unique()):
    df_pred_sub = df_pred[df_pred.img == img]
    df_label_sub = df_label[df_label.img == img].reset_index()

    # Calculate IoU for each pair of prediction and label bounding boxes in a vectorized manner
    pred_boxes = df_pred_sub[['xmin', 'ymin', 'xmax', 'ymax']].values
    label_boxes = df_label_sub[['xmin', 'ymin', 'xmax', 'ymax']].values

    for idx, pred_box in enumerate(pred_boxes):
        ious=[]
        for label_box in label_boxes:
            ious.append(box_iou(pred_box, label_box))
            # Update the 'iou' column with the maximum IoU for each prediction
        max_ious = np.max(ious)
        max_ious_index = ious.index(max_ious)
        # print(ious)
        # print(max_ious)
        # print(idx)
        # print(df_pred_sub[0])
        # print(df_pred_sub.index[idx])
        df_pred_iou.at[df_pred_sub.index[idx], 'iou'] = max_ious
        df_pred_iou.at[df_pred_sub.index[idx], 'label'] = df_label_sub['class'][max_ious_index]
        # print(df_label_sub['class'][max_ious_index])

# Filter out predictions based on the IoU threshold
df_pred_lowIou = df_pred_iou[df_pred_iou['iou'] < iou_th]
df_pred_iou = df_pred_iou[df_pred_iou['iou'] >= iou_th]

# Remove the 'iou' column, if needed
# df_pred_iou = df_pred_iou.drop(columns=['iou'])

100%|██████████| 733/733 [00:03<00:00, 233.44it/s]


In [44]:
df_pred_iou[df_pred_iou.label==df_pred_iou['class']]

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax,iou,label
500,2a69f2ad271728c99a849d679745492e#201201170710_...,3,0.298788,0.305005,79,215,5,111,0.898882,3.0
501,2a69f2ad271728c99a849d679745492e#201201170710_...,3,0.348404,0.365612,79,215,5,113,0.915372,3.0
506,2a69f2ad271728c99a849d679745492e#201201170710_...,3,0.266675,0.281709,119,245,5,104,0.807692,3.0
547,390b3133d417e8660afd01c18abd1a19#201102271650_...,7,0.283288,0.293799,277,315,209,261,0.404211,7.0
802,7cadb2ec9cb2e59616e76fe98e85b43c#201105051700_...,2,0.458702,0.505771,400,522,113,195,0.774780,2.0
803,7cadb2ec9cb2e59616e76fe98e85b43c#201105051700_...,2,0.296200,0.413171,451,533,162,261,0.560945,2.0
916,c969b3d66925a2387f01fb0a6a9ae06c#201104061510_...,9,0.541753,0.560408,464,516,154,212,0.826548,9.0
960,c969b3d66925a2387f01fb0a6a9ae06c#201104061510_...,9,0.266586,0.380013,519,587,281,332,0.814145,9.0
982,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,9,0.264186,0.314574,311,385,180,243,0.953846,9.0
984,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,9,0.345403,0.414065,308,364,194,257,0.909091,9.0


In [36]:
df_pred_iou.to_csv('hard_test_highIou.csv')

In [40]:
sorted(df_label['class'].unique())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [45]:
df_pred_lowIou

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax,iou,label
47,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.309022,0.310376,488,544,180,219,0.092593,5.0
83,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.451213,0.455555,585,639,194,289,0.248496,5.0
98,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.462216,0.462992,533,587,185,236,0.033438,5.0
99,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.491953,0.521484,85,156,28,76,0.000000,5.0
146,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.261220,0.277229,562,611,56,116,0.078235,8.0
192,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.355440,0.358096,80,164,145,236,0.243205,5.0
209,07baf895b711927a078c12eb6deb88b0#201011270630_...,9,0.364675,0.448898,558,615,108,153,0.000000,5.0
270,1bff883d55ca0bbb67351f8169f76914#201011150930_...,3,0.517595,0.518879,306,359,173,219,0.007850,5.0
277,1bff883d55ca0bbb67351f8169f76914#201011150930_...,3,0.310161,0.312127,51,106,120,180,0.000000,5.0
279,1bff883d55ca0bbb67351f8169f76914#201011150930_...,3,0.488609,0.489466,394,431,46,85,0.000000,5.0


In [47]:
imgPath = 'C:\\Users\\leofi\\OneDrive - Universidade de Lisboa\\Documents\\GitHub\\masters\\Data\\fc2015_yolo_uc_hard\\test\\images'
buildMontage(df_pred_lowIou, imgPath, show=True)

100%|██████████| 58/58 [00:00<00:00, 112.73it/s]


[array([[[  5,  10,  19],
         [ 32,   7,  11],
         [123,  74,  72],
         ...,
         [255, 255, 198],
         [199, 193, 152],
         [ 33,  15,   8]],
 
        [[  0,   0,   7],
         [ 33,  12,  15],
         [174, 137, 133],
         ...,
         [255, 255, 196],
         [198, 192, 149],
         [ 32,  14,   7]],
 
        [[ 44,  40,  51],
         [ 68,  45,  49],
         [ 50,  12,   8],
         ...,
         [255, 255, 195],
         [197, 191, 148],
         [ 32,  15,   6]],
 
        ...,
 
        [[  0,   1,   2],
         [ 14,   6,  17],
         [ 27,  17,  30],
         ...,
         [ 84, 119,  55],
         [ 31,  53,  25],
         [  0,   5,   8]],
 
        [[  0,   1,   2],
         [ 14,   6,  17],
         [ 27,  16,  32],
         ...,
         [ 37,  63,  27],
         [  0,  13,   7],
         [  0,   1,  20]],
 
        [[  0,   1,   2],
         [ 14,   6,  17],
         [ 28,  17,  33],
         ...,
         [ 37,  56,  39],
  